In [25]:
import numpy as np

# 랜덤시드 고정시키기
np.random.seed(3)

from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [86]:
# 데이터셋 불러오기
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/Users/chanju/github/mask_filtering_project/2주차/이찬주/LeeChanJu/data/train',
        target_size=(28, 28),
        batch_size=3,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        '/Users/chanju/github/mask_filtering_project/2주차/이찬주/LeeChanJu/data/val',
        target_size=(28, 28),    
        batch_size=32,
        class_mode='categorical')

Found 1291 images belonging to 2 classes.
Found 142 images belonging to 2 classes.


filter out corrupted images

In [87]:
import os

num_skipped = 0
for folder_name in ("with_mask", "without_mask"):
    folder_path = os.path.join("/Users/chanju/github/mask_filtering_project/2주차/이찬주/LeeChanJu/data/train", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname) # PetImages/Cat/63.jpg
        try:
            fobj = open(fpath, "rb")
            # fobj.peek(10)은 10바이트를 가져오는데, 여기서는 보통
            # 헤더를 읽기 위해 전체를 가져온다고 생각해도 무방합니다.
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print("Deleted %d images" % num_skipped)

Deleted 0 images


In [88]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D

In [89]:
# 모델 구성하기
model = Sequential()
model.add(Conv2D(64, kernel_size=(3,3), padding='same', input_shape=(28,28,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, kernel_size=(3,3),padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(256, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

In [90]:
model.summary(0)

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_31 (Conv2D)           (None, 28, 28, 64)        1792      
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 14, 14, 128)       73856     
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 7, 7, 256)         295168    
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 3, 3, 256)         0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 2304)            

In [91]:
# 모델 엮기
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [92]:
# 모델 학습시키기
# 모델 학습시키기
model.fit_generator(
    train_generator,
    steps_per_epoch=20,
    epochs=200,
    validation_data=test_generator,
    validation_steps=10,
)

Epoch 1/200
20/20 [==============================] - 2s 60ms/step - loss: 0.7284 - accuracy: 0.5081 - val_loss: 0.7169 - val_accuracy: 0.5000
Epoch 2/200
20/20 [==============================] - 0s 22ms/step - loss: 0.7583 - accuracy: 0.2961
Epoch 3/200
20/20 [==============================] - 0s 13ms/step - loss: 0.7041 - accuracy: 0.6554
Epoch 4/200
20/20 [==============================] - ETA: 0s - loss: 0.6839 - accuracy: 0.65 - 0s 13ms/step - loss: 0.6840 - accuracy: 0.6505
Epoch 5/200
20/20 [==============================] - 0s 22ms/step - loss: 0.6217 - accuracy: 0.8752
Epoch 6/200
20/20 [==============================] - 0s 16ms/step - loss: 0.3693 - accuracy: 0.8960
Epoch 7/200
20/20 [==============================] - 0s 12ms/step - loss: 0.2031 - accuracy: 0.9475
Epoch 8/200
20/20 [==============================] - 0s 21ms/step - loss: 0.5444 - accuracy: 0.8310
Epoch 9/200
20/20 [==============================] - 0s 12ms/step - loss: 0.1415 - accuracy: 0.9733
Epoch 10/200
20/

20/20 [==============================] - 0s 12ms/step - loss: 0.0427 - accuracy: 0.9810
Epoch 78/200
20/20 [==============================] - 0s 17ms/step - loss: 0.0258 - accuracy: 1.0000
Epoch 79/200
20/20 [==============================] - 0s 12ms/step - loss: 0.0112 - accuracy: 1.0000
Epoch 80/200
20/20 [==============================] - 0s 13ms/step - loss: 0.0329 - accuracy: 1.0000
Epoch 81/200
20/20 [==============================] - 1s 29ms/step - loss: 0.0057 - accuracy: 1.0000
Epoch 82/200
20/20 [==============================] - 0s 17ms/step - loss: 0.0280 - accuracy: 0.9948
Epoch 83/200
20/20 [==============================] - 0s 14ms/step - loss: 0.1275 - accuracy: 0.9609
Epoch 84/200
20/20 [==============================] - 0s 13ms/step - loss: 0.0788 - accuracy: 0.9593
Epoch 85/200
20/20 [==============================] - 0s 15ms/step - loss: 0.0169 - accuracy: 1.0000
Epoch 86/200
20/20 [==============================] - 0s 14ms/step - loss: 0.0347 - accuracy: 0.9861
Epo

20/20 [==============================] - 0s 14ms/step - loss: 0.0059 - accuracy: 1.0000
Epoch 158/200
20/20 [==============================] - 0s 13ms/step - loss: 0.0021 - accuracy: 1.0000
Epoch 159/200
20/20 [==============================] - 0s 12ms/step - loss: 0.1011 - accuracy: 0.9643
Epoch 160/200
20/20 [==============================] - 0s 11ms/step - loss: 0.0097 - accuracy: 1.0000
Epoch 161/200
20/20 [==============================] - 0s 12ms/step - loss: 0.0076 - accuracy: 1.0000
Epoch 162/200
20/20 [==============================] - 1s 37ms/step - loss: 0.0260 - accuracy: 0.9759
Epoch 163/200
20/20 [==============================] - 1s 29ms/step - loss: 0.1461 - accuracy: 0.9545
Epoch 164/200
20/20 [==============================] - 0s 20ms/step - loss: 0.0744 - accuracy: 0.9604
Epoch 165/200
20/20 [==============================] - 0s 16ms/step - loss: 0.0937 - accuracy: 0.9752
Epoch 166/200
20/20 [==============================] - 0s 12ms/step - loss: 0.0042 - accuracy: 1

In [93]:
# 모델 평가하기
print("-- Evaluate --")

-- Evaluate --


In [94]:
scores = model.evaluate_generator(
            test_generator, 
            steps = 5)

In [95]:
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

accuracy: 97.18%


In [96]:
# 모델 예측하기
print("-- Predict --")

output = model.predict(
            test_generator, 
            steps = 5)

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

print(output)

-- Predict --
[[1.000 0.000]
 [1.000 0.000]
 [0.000 1.000]
 [1.000 0.000]
 [1.000 0.000]
 [1.000 0.000]
 [1.000 0.000]
 [1.000 0.000]
 [1.000 0.000]
 [1.000 0.000]
 [1.000 0.000]
 [0.000 1.000]
 [1.000 0.000]
 [0.000 1.000]
 [1.000 0.000]
 [1.000 0.000]
 [1.000 0.000]
 [1.000 0.000]
 [0.000 1.000]
 [1.000 0.000]
 [1.000 0.000]
 [1.000 0.000]
 [0.000 1.000]
 [0.000 1.000]
 [0.000 1.000]
 [1.000 0.000]
 [0.000 1.000]
 [1.000 0.000]
 [0.000 1.000]
 [0.000 1.000]
 [1.000 0.000]
 [1.000 0.000]
 [1.000 0.000]
 [1.000 0.000]
 [0.000 1.000]
 [1.000 0.000]
 [1.000 0.000]
 [1.000 0.000]
 [1.000 0.000]
 [1.000 0.000]
 [0.000 1.000]
 [0.992 0.007]
 [0.000 1.000]
 [0.976 0.027]
 [1.000 0.000]
 [0.000 1.000]
 [0.000 1.000]
 [1.000 0.000]
 [0.000 1.000]
 [0.000 1.000]
 [0.000 1.000]
 [0.000 1.000]
 [0.000 1.000]
 [0.000 1.000]
 [1.000 0.000]
 [0.000 1.000]
 [0.001 0.999]
 [0.000 1.000]
 [0.000 1.000]
 [0.000 1.000]
 [0.000 1.000]
 [1.000 0.000]
 [1.000 0.000]
 [1.000 0.000]
 [0.001 0.999]
 [0.000 1.0